# Module 2 Summative Lab

## Introduction

For today's section, we're going to work on a single big lab to apply everything we've learned in Module 2!

## About This Lab

A quick note before getting started--this lab isn't like other labs you seen so far. This lab is meant to take ~8 hours to complete, so it's much longer and more challenging that the average labs you've seen so far. If you feel like this lab is challenging or that you might be struggling a bit, don't fret--that's by design! With everything we've learned about Web Scraping, APIs, and Databases, the best way to test our knowledge of it is to build something substantial! 

## The Project

In this lab, we're going to make use of everything we've learned about APIs, databases, and Object-Oriented Programming to **_Extract, Transform, and Load_** (or **_ETL_**, for short) some data from a SQL database into a MongoDB Database. 

You'll find a database containing information about soccer teams and the matches they've played in the file `database.sqlite`. For this project, our goal is to get the data we think is important from this SQL database, do some calculations and data transformation, and then store everything in a MongoDB database. 

Let's get into the specifics of this project.

### The Goal

Start by examining the data dictionary for the SQL database we'll be working with, which comes from this [kaggle page](https://www.kaggle.com/laudanum/footballdelphi).  Familiarize yourself with the tables it contains, and what each column means. We'll be using this database to get data on each soccer team, calculate some summary statistics, and then store each in a MongoDB database. 

Upon completion of this lab, each unique team in this dataset should have a record in the MongoDB instance containing the following information:

* The name of the team
* The total number of goals scored by the team during the 2011 season
* The total number of wins the team earned during the 2011 season
* A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly)
* The team's win percentage on days where it was raining during games in the 2011 season. 

#### Getting the Weather Data

Note that for this last calculation, you'll need to figure out if it was raining or not during the game. The database itself does not contain this information, but it does contain the date on which the game was played. For this, you'll need to use the [DarkSky API](https://darksky.net/dev) to get the historical weather data for that day. Note that each game is played in a different location, and this information is not contained in our SQL database. However, the teams in this database are largely german, so go ahead and just use the weather in Berlin, Germany as a proxy for this information. If it was raining in Berlin on the day the game was played, count that as rain game--**_you do not need to try and figure out the actual weather at each game's location, because we don't have that information!_**

#### NOTE: The DarkSky API is limited to 1000 free API calls a day, so be sure to test your model on very small samples. Otherwise, you'll hit the rate limit!

## Project Architecture

Unlike previous labs, this lab is more open-ended, and will require you to make design decisions and plan out your strategy for building a system with this many working parts. However, **_using Object-Oriented Programming is a requirement for this project--you must create at least 2 separate, well structured classes in your solution!_** Although it may seem easier to "just start coding", this is a classic beginner's mistake. Instead, think about separating out the different functionalities you'll need to reach your goal, and then build classes to handle each. For instance, at minimum, you'll need to:

* Query the SQL database
* Calculate summary statistics
* Get the weather data from the DarkSky API
* Load the data into MongoDB

We **_strongly recommend_** you consider creating separate classes for handling at least some of these of these tasks.  Be sure to plan the inputs, outputs, and methods for each class before you begin coding! 

**_NOTE:_** We have provided some empty classes below. You are welcome to delete them and use a different architecture for this project if you so choose.  You do not have to use each of them, they are just there to give you an idea of what you could sorts of classes you may want to consider using.

### Rapid Prototyping and Refactoring

It's totally okay to try to get a task working without using OOP. For instance, when experimenting with the DarkSky API for getting historical weather data, it makes sense to just write the code in the cells and rapidly iterate until you get it all working. However, once you get it working, you're not done--you should then **_Refactor_** your code into functions or classes to make your code more modular, reusable, understandable, and maintainable! 

In short--do what you need to do to get each separate piece of functionality working, and then refactor it into a class after you've figured it out!

### Some Final Advice

You haven't built anything this big or complex thus far, so you may not yet fully realize how much trial and error goes into it. If your code keeps breaking, resist the urge to get frustrated, and just keep working. Software development is an iterative process!  No one writes perfect code that works the first time for something this involved. You're going to run into _a lot_ of small errors in this project, right up until the point where it just works, and then you're done! However, you can reduce these errors by planning out your code, and thinking about how all of the pieces fit together before you begin coding. Once you have some basic understanding of how it all will work, then you'll know what you need to build, and then all that is left is to build it!

In short:

* Plan ahead--you'll thank yourself later!
* Errors and broken code aren't bad, they're normal. 
* Keep working, and stay confident--you can do this!

Good luck--we look forward to seeing your completed project!

In [1]:
# We've scaffolded your classes in the provided lib.py file

from lib import WeatherGetter
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np


%load_ext autoreload

In [2]:
# connecting with database file and storing it as a pandas dataframe to perform required analysis

con = sqlite3.connect('database.sqlite')
cur = con.cursor()
cur.execute("""SELECT * FROM Matches WHERE season == 2011""")

df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
df.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


In [3]:
# only subset those teams which played in germany and store in dataframe, also look at the shape

df = df[(df.Div == 'D1')|(df.Div=='D2')]
df.shape

(612, 9)

In [4]:
# to add information on rainy days we will look for length of unique dates we have to work with

dates = list(df.Date.unique())
len(dates)

136

In [5]:
# we create a instance of our weathegetter object and use it to create a list of boolean value on rainy days

weather = WeatherGetter()
weather_report = weather.test_many(dates)

In [6]:
""" Optional cell for saving the weather_report as a JSON file which can be
    used for analysis. This is meant to prevent making pointless requests
    to DarkSky API"""

# import json
# with open('weather.json', 'w') as fp:
#     json.dump(weather_report, fp)

' Optional cell for saving the weather_report as a JSON file which can be\n    used for analysis. This is meant to prevent making pointless requests\n    to DarkSky API'

In [7]:
""" create a list called rainy_day which holds boolean value of whether it rained on a particular day from the date
    columns of our dataframe. We then add that list as a new column in our dataframe.
"""

rainy_day = []
for date in df.Date:
    rainy_day.append(weather_report[date])

df['Rainy_Day'] = rainy_day
df.head() #testing to see if it worked

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Rainy_Day
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,True
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,False
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,False
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,False
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,False


In [8]:
""" We need to add couple more new columns, regarding the teams overall wins, losses and total matches played
    We also create another column rain_win and rain_loss and initialise it as 0
"""

df['wins'] = df['FTR'].map(lambda x: 1 if x == 'H' else 0)
df['loss'] = df['FTR'].map(lambda x: 1 if x == 'A' else 0)
df['totalMatch'] = 1    # since they played only one match on that corresponding date
df['rain_wins'] = 0     
df['rain_loss'] = 0

# Now we change values of rain_win & rain_loss on those rows where there was a rain and they either won or lost
# We will achieve this by first subsetting the dataframe to obtain only those where it rained or not rained

df[(df.wins == 1)&(df.Rainy_Day == True)] = df[(df.wins == 1)&(df.Rainy_Day == True)].assign(rain_wins = 1)
df[(df.loss == 1)&(df.Rainy_Day == True)] = df[(df.loss == 1)&(df.Rainy_Day == True)].assign(rain_loss = 1)

df.head(4)


,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Rainy_Day,wins,loss,totalMatch,rain_wins,rain_loss
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,True,0,1,1,0,1
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,False,0,1,1,0,0
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,False,0,1,1,0,0
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,False,1,0,1,0,0


In [9]:
""" Now we can just use the groupby function to group the data on hometeam. We can sum values for wins, loss,
    total goals and other columns of interest. We will check the result to see if we got it right
"""

match_sum = df.groupby('HomeTeam').sum()
match_sum.head(4)

,Match_ID,Season,FTHG,FTAG,Rainy_Day,wins,loss,totalMatch,rain_wins,rain_loss
HomeTeam,,,,,,,,,,
Aachen,24768,34187,15,24,5.0,4,7,17,1,0
Augsburg,22397,34187,20,19,7.0,6,4,17,4,2
Bayern Munich,22898,34187,49,6,4.0,14,2,17,3,1
Bochum,24898,34187,23,23,5.0,7,7,17,1,3


In [10]:
# The only thing missing is rain_win percentage. We create a function to help calcualte them
# Now we calculate rain_win percentage

def win_percentage(win, loss):
    total = win + loss
    return ((win/total)*100).round(2)

# apply the function to all rows of the match_sum dataframe

match_sum = match_sum.assign(rainWin_percent = lambda x: win_percentage(x['rain_wins'], x['rain_loss']))                           
match_sum.head()


,Match_ID,Season,FTHG,FTAG,Rainy_Day,wins,loss,totalMatch,rain_wins,rain_loss,rainWin_percent
HomeTeam,,,,,,,,,,,
Aachen,24768,34187,15,24,5.0,4,7,17,1,0,100.00
Augsburg,22397,34187,20,19,7.0,6,4,17,4,2,66.67
Bayern Munich,22898,34187,49,6,4.0,14,2,17,3,1,75.00
Bochum,24898,34187,23,23,5.0,7,7,17,1,3,25.00
Braunschweig,24745,34187,21,15,4.0,6,3,17,2,1,66.67


In [11]:
import pymongo

class MongoHandler(object):
    
    def __init__(self):
        self.myclient = pymongo.MongoClient('mongodb://localhost:27017')
        self.db = self.myclient['Football']
        self.coll = self.db['Summary_2011']
    
    def format_data(self, team_name, goals_2011, wins_2011, rain_win_pct):
        data = {"team_name": team_name,
               "goals_2011": goals_2011,
               "wins_2011": wins_2011,
               "rain_win_pct": rain_win_pct}
        return data
    
    def insert_record(self, record):
        return self.coll.insert_one(record)  

In [12]:
def viz_bar(row):
    height = [row['wins'],row['loss']]
    bars = ('Wins', 'Losses')
    y_pos = np.arange(len(bars))   
    viz = plt.bar(bars, height)
    return viz
    

In [13]:
# Bar chart could only be inserted as a matplotlib object value
# So I have removed it from final result

da = MongoHandler()
for row in match_sum.index:
    record = da.format_data(row, match_sum.loc[row]['FTHG'], match_sum.loc[row]['wins'], match_sum.loc[row]['rainWin_percent'])
    print(record)
    da.insert_record(record)
    

{'team_name': 'Aachen', 'goals_2011': 15.0, 'wins_2011': 4.0, 'rain_win_pct': 100.0}
{'team_name': 'Augsburg', 'goals_2011': 20.0, 'wins_2011': 6.0, 'rain_win_pct': 66.67}
{'team_name': 'Bayern Munich', 'goals_2011': 49.0, 'wins_2011': 14.0, 'rain_win_pct': 75.0}
{'team_name': 'Bochum', 'goals_2011': 23.0, 'wins_2011': 7.0, 'rain_win_pct': 25.0}
{'team_name': 'Braunschweig', 'goals_2011': 21.0, 'wins_2011': 6.0, 'rain_win_pct': 66.67}
{'team_name': 'Cottbus', 'goals_2011': 18.0, 'wins_2011': 4.0, 'rain_win_pct': 33.33}
{'team_name': 'Dortmund', 'goals_2011': 44.0, 'wins_2011': 14.0, 'rain_win_pct': 80.0}
{'team_name': 'Dresden', 'goals_2011': 30.0, 'wins_2011': 8.0, 'rain_win_pct': 100.0}
{'team_name': 'Duisburg', 'goals_2011': 23.0, 'wins_2011': 8.0, 'rain_win_pct': 50.0}
{'team_name': 'Ein Frankfurt', 'goals_2011': 38.0, 'wins_2011': 11.0, 'rain_win_pct': 100.0}
{'team_name': 'Erzgebirge Aue', 'goals_2011': 19.0, 'wins_2011': 5.0, 'rain_win_pct': 33.33}
{'team_name': 'FC Koln', 'goal

# Summary

In this lab, we dug deep and used everything we've learned so far about python programming, databases, HTTP requests and API calls to ETL data from a SQL database into a MongoDB instance!